We will use `pygmsh` to create a `box`.
The `box` object can only be initialized in an axis-aligned orientation.
That means its edges will go along the (x, y, z) directions and not at a 45 degree angle, for instance.
In a minute we will do some rotations with `scipy`.
> You will get a warning about builtin ctypes, you can ignore this.

In [21]:
import pygmsh
import numpy as np

with pygmsh.occ.Geometry() as geom:  # initialize the geometry kernel
    x0 = np.array([-1, -1, 0])       # define the lower-left-most point
    dims = np.array([2, 1, 0.25])    # define the extent in each dimension (x, y, z)
    box = geom.add_box(x0, dims)     # add the box to the scene
    mesh = geom.generate_mesh()      # generate the mesh from the defined shapes
    
print(mesh)

<meshio mesh object>
  Number of points: 174
  Number of cells:
    line: 64
    triangle: 344
    tetra: 424
    vertex: 8


C:\Users\Wes\AppData\Roaming\Python\Python37\site-packages\numpy\ctypeslib.py:523: RuntimeWarning: A builtin ctypes object gave a PEP3118 format string that does not match its itemsize, so a best-guess will be made of the data type. Newer versions of python may behave correctly.
  return array(obj, copy=False)


We use `print` to get a summary of the generated mesh.
To actually visualize it, we can use an interactive `k3d` plot.
That means you can pan, rotate, zoom, etc after the plot is rendered.
These features will help with designing 3D geometries.

In [22]:
import k3d

plot = k3d.plot()                                                   # create an interactive k3d plot
plt_mesh = k3d.mesh(mesh.points.astype(np.float32),                 # add a mesh to the plot using the mesh `points`...
                    mesh.cells_dict['triangle'].astype(np.uint32),  # ...and the `triangle` cells defined in the mesh...
                    wireframe=True, color=0x000000)                 # ...and render it in "wireframe" to show the triangles
plot += plt_mesh                                                    # add the `plt_mesh` element to the k3d plot
plot.display()                                                      # display the k3d plot

Output()

Let's use `scipy` to compute some rotations for us.
The package `pygmsh` does rotations using the following information:
* Rotation origin (the point about which the rotation will occur)
* Rotation axis (the axis about which the object will rotate)
* Rotation angle (the magnitude of the rotation)

We can get these from `scipy` in the form of a `rotvec` or rotation vector.
The direction of the rotation vector defines the rotation and the magnitude defines the angle in radians.

In [24]:
from scipy.spatial.transform import Rotation

R = Rotation.from_euler('x', np.pi/4)  # create a `Rotation` object that rotates about `x` by 45 degrees (in radians!)
axis = R.as_rotvec()                   # compute the rotation axis
angle = np.linalg.norm(axis)           # calculate the rotation angle using numpy to get the norm

print(axis, angle)

[0.78539816 0.         0.        ] 0.7853981633974484


Here you can see that the rotation axis goes along the x axis as we requested.
Let's see how this works in `pygmsh`...

In [25]:
with pygmsh.occ.Geometry() as geom:
    x0 = np.array([-1, -1, 0])     # create the box as before...
    dims = np.array([2, 1, 0.25])
    box = geom.add_box(x0, dims)
    
    box = geom.rotate(box,        # the rotation takes the object as input...
                      [0, 0, 0],  # ...we'll do the rotation around the origin, but we could pick any point
                      angle,      # ...define the rotation angle as we computed above...
                      axis)       # ...define the rotation axis as we computed above

    mesh = geom.generate_mesh()
    
plot = k3d.plot()
plt_mesh = k3d.mesh(mesh.points.astype(np.float32),
                    mesh.cells_dict['triangle'].astype(np.uint32),
                    wireframe=True, color=0x000000)
plot += plt_mesh
plot.display()

C:\Users\Wes\AppData\Roaming\Python\Python37\site-packages\numpy\ctypeslib.py:523: RuntimeWarning: A builtin ctypes object gave a PEP3118 format string that does not match its itemsize, so a best-guess will be made of the data type. Newer versions of python may behave correctly.
  return array(obj, copy=False)


Output()

We can also define multiple rotations using the Euler angles, such as 45 degrees about x and 30 degrees around y.

In [26]:
R = Rotation.from_euler('xy', [np.pi/4, np.pi/6])
axis = R.as_rotvec()
angle = np.linalg.norm(axis)

with pygmsh.occ.Geometry() as geom:
    x0 = np.array([-1, -1, 0])     # create the box as before...
    dims = np.array([2, 1, 0.25])
    box = geom.add_box(x0, dims)
    
    box = geom.rotate(box,        # the rotation takes the object as input...
                      [0, 0, 0],  # ...we'll do the rotation around the origin, but we could pick any point
                      angle,      # ...define the rotation angle as we computed above...
                      axis)       # ...define the rotation axis as we computed above

    mesh = geom.generate_mesh()
    
plot = k3d.plot()
plt_mesh = k3d.mesh(mesh.points.astype(np.float32),
                    mesh.cells_dict['triangle'].astype(np.uint32),
                    wireframe=True, color=0x000000)
plot += plt_mesh
plot.display()

C:\Users\Wes\AppData\Roaming\Python\Python37\site-packages\numpy\ctypeslib.py:523: RuntimeWarning: A builtin ctypes object gave a PEP3118 format string that does not match its itemsize, so a best-guess will be made of the data type. Newer versions of python may behave correctly.
  return array(obj, copy=False)


Output()